In [1]:

from gpkit import Model, Variable
from gpkit.constraints.tight import Tight
from gpkit.constraints.bounded import Bounded
from gpkit import ureg

from BasicRocketComponents import Motor, Tube

​

In [2]:

class Rocket(Model):
    
    def setup(self):
        
        constraints = []
        components = []
        
        
        #define all components
        
        nc   = self.nc   = NoseCone()
        body = self.body = Tube()
        fins = self.fins = Fins()
        
        av = self.av = Avionics()
        payload = self.payload = Payload()
        
        recovery = self.recovery = Recovery()
        
        mainMotor = self.mainMotor = MainMotor()
        boosters  = self.boosters  = Booster()
        
        
        
        components += [nc, body, fins, mainMotor, boosters]
        
        
        
        

In [55]:
class MainMotor(Model):
    
    def setup(self):
        
        constraints = []
        components = self.components = []
        m = self.m = Variable("m", "kg", "Mass of Main Motor")
        
        motor = self.motor = Motor()
        components += [motor]
        
        constraints += [motor.m_dry >= 0.7*self.m] #based on most rockets on thrustcurve.org
        constraints += [Tight([motor.F_avg >= 750*ureg.N])]
        constraints += [Tight([motor.t_burn >= 20*ureg.s])]
        
        constraints += [Tight([motor.c <= 1800*ureg.m/ureg.s])] #based on most rockets on thrustcurve.org
                                    
        
        
        return [components, constraints, Tight([self.m >= sum(comp.m for comp in components)])]
    
class Booster(Model):
    
    def setup(self):
        
        constraints = []
        components = self.components = []
        m = self.m = Variable("m", "kg", "Mass of all boosters")
        
        
        num = Variable("num", "", "Number of boosters", integer = True)
        
        constraints += [num>=2]
        
        motor = self.motor = Motor()
        components += [motor]
        
        #generic bounds, computing the bounds based on the vehicle mass is done in the Rocket() class. 
        constraints += [motor.m_dry >= 0.1*ureg.kg]
        constraints += [Tight([motor.F_avg >= 10*ureg.N])]
        constraints += [Tight([motor.t_burn >= 1*ureg.s])]
        
        constraints += [Tight([motor.c <= 3000*ureg.m/ureg.s])]
                                    
        #in this case, we define the mass constraint explicity
        constraints += [Tight([self.m >= num*motor.m])]
        
        return [components, constraints]

In [56]:
m = Motor()

In [57]:
mm = MainMotor()

In [58]:
mm

[[[gpkit.PosynomialInequality(m_MainMotor.5/Motor >= m_{dry}_MainMotor.5/Motor + m_{prop}_MainMotor.5/Motor),
   [gpkit.PosynomialInequality(T_{avg}_MainMotor.5/Motor <= c_MainMotor.5/Motor*m_{prop}_MainMotor.5/Motor*t_{burn}_MainMotor.5/Motor**-1)],
   [gpkit.MonomialEquality(I_t_MainMotor.5/Motor = c_MainMotor.5/Motor*m_{prop}_MainMotor.5/Motor)],
   [gpkit.PosynomialInequality(I_t_MainMotor.5/Motor >= c_MainMotor.5/Motor*m_{prop}_MainMotor.5/Motor)]]],
 [gpkit.PosynomialInequality(m_{dry}_MainMotor.5/Motor >= 0.7*m_MainMotor.5),
  [gpkit.PosynomialInequality(T_{avg}_MainMotor.5/Motor >= 750)],
  [gpkit.PosynomialInequality(t_{burn}_MainMotor.5/Motor >= 20)],
  [gpkit.PosynomialInequality(c_MainMotor.5/Motor <= 1.8e+03)]],
 [gpkit.PosynomialInequality(m_MainMotor.5 >= m_MainMotor.5/Motor)]]

In [59]:
opt = Model(mm.m, Bounded(mm))

In [60]:
sol = opt.solve()

Using solver 'cvxopt'
Solving for 8 variables.
Solving took 0.0532 seconds.


In [61]:
print(sol.summary())


Cost
----
 27.78 [kg]

Free Variables
--------------
         | MainMotor.5
       m : 27.78    [kg]  Mass of Main Motor

         | MainMotor.5/Motor
     I_t : 1.5e+04  [N·s] Total impulse of motor
 T_{avg} : 750      [N]   Average force of motor
       c : 1800     [m/s] Exhaust speed of motor
       m : 27.78    [kg]  Total mass of motor
 m_{dry} : 19.44    [kg]  Dry Mass of Motor
m_{prop} : 8.333    [kg]  Propellant Mass of Motor
t_{burn} : 20       [s]   Burn Time

Tightest Constraints
--------------------
       | MainMotor.5
  +3.3 : m_ >= m_/Motor
  +2.3 : m_{dry}_/Motor >= 0.7*m_
    +1 : T_{avg}_/Motor >= 750

       | MainMotor.5/Motor
  +3.3 : m_ >= m_{dry}_ + m_{prop}_
    +1 : T_{avg}_ <= c_*m_{prop}_*t_{burn}_**-1



In [45]:
booster = Booster()

In [46]:
sol = Model(booster.m, Bounded(booster)).solve()

Using solver 'cvxopt'
Solving for 9 variables.
Solving took 0.0615 seconds.


In [47]:
print(sol.summary())


Cost
----
 0.2067 [kg]

Free Variables
--------------
         | Booster.3
       m : 0.2067    [kg]  Mass of all boosters
     num : 2               Number of boosters

         | Booster.3/Motor
     I_t : 10        [N·s] Total impulse of motor
 T_{avg} : 10        [N]   Average force of motor
       c : 3000      [m/s] Exhaust speed of motor
       m : 0.1033    [kg]  Total mass of motor
 m_{dry} : 0.1       [kg]  Dry Mass of Motor
m_{prop} : 0.003333  [kg]  Propellant Mass of Motor
t_{burn} : 1         [s]   Burn Time

Tightest Constraints
--------------------
       | Booster.3/Motor
  +1.1 : I_t_ = c_*m_{prop}_
  +1.1 : I_t_ >= c_*m_{prop}_
    +1 : m_ >= m_{dry}_ + m_{prop}_

       | Booster.3
    +1 : m_ >= m_/Motor*num_
    +1 : num_ >= 2

